##### ✅ 코루틴 사용하기
Kotlin 코루틴에서 자주 사용하는 기능들을 정리한 내용입니다.

---

#### 1. runBlocking
- 코루틴을 블로킹 방식으로 실행
- 테스트나 진입 지점에서 주로 사용

```kotlin
runBlocking {
    println("Start")
    delay(1000)
    println("End")
}
```

---

#### 2. launch / async / withContext
| 함수 | 설명 |
|------|------|
| launch | 결과가 없는 비동기 작업 수행 |
| async | 결과가 필요한 비동기 작업 수행 (await()로 값 반환) |
| withContext | 특정 Dispatcher로 context 전환 |

---

#### 3. Flow & 연산자
| 함수 | 설명 |
|------|------|
| flow | 콜드 스트림 생성 |
| collect | 스트림의 값을 수집 |
| collectLatest | 가장 최신 값만 처리, 이전 값은 취소됨 |
| map / filter | 값 변환 / 조건 필터링 |
| flatMapLatest / switchMap | 가장 최신 flow만 유지 |
| debounce | 일정 시간 간격 내 연속 입력 무시 |
| retry / catch | 재시도 및 예외 처리 |

---

#### 4. Android에서 자주 쓰는 CoroutineScope
| 스코프 | 설명 |
|--------|------|
| viewModelScope | ViewModel 내 생명주기와 연결된 코루틴 실행 |
| lifecycleScope | Activity/Fragment 생명주기 내 안전한 실행 |
| repeatOnLifecycle | 특정 상태(Lifecycle.State.STARTED 등)에서만 수집 수행 |

---

#### 📌 실무 예시 1 - 사용자 입력 처리 (검색창)
```kotlin
val queryFlow = MutableStateFlow("")

val resultsFlow = queryFlow
    .debounce(300)
    .filter { it.isNotBlank() }
    .flatMapLatest { keyword -> searchApi(keyword) }

lifecycleScope.launch {
    resultsFlow.collectLatest { results ->
        updateUI(results)
    }
}
```

---

# #### 📌 실무 예시 2 - ViewModel에서 비동기 API 호출
```kotlin
fun fetchData() {
    viewModelScope.launch {
        val result = withContext(Dispatchers.IO) {
            repository.loadData()
        }
        _uiState.value = result
    }
}
```

---

#### 📌 실무 예시 3 - suspend 함수 정의 및 사용

```kotlin
suspend fun fetchUser(): User {
    delay(500)
    return User("kim", 30)
}

fun example() {
    CoroutineScope(Dispatchers.IO).launch {
        val user = fetchUser()
        println("user=$user")
    }
}
```

---

#### 📌 실무 예시 4 - flowOf를 활용한 데이터 스트림

```kotlin
val numbers = flowOf(1, 2, 3, 4, 5)

runBlocking {
    numbers.collect { println(it) }
}
```

---

#### 📌 실무 예시 5 - channel을 사용한 코루틴 간 통신

```kotlin
val channel = Channel<Int>()

CoroutineScope(Dispatchers.Default).launch {
    repeat(5) {
        channel.send(it)
    }
    channel.close()
}

CoroutineScope(Dispatchers.Default).launch {
    for (value in channel) {
        println("Received: $value")
    }
}
```

---

#### 📌 실무 예시 6 - SharedFlow와 StateFlow

```kotlin
val sharedFlow = MutableSharedFlow<String>()
val stateFlow = MutableStateFlow("Initial")

// 수집
CoroutineScope(Dispatchers.Main).launch {
    sharedFlow.collect { println("Shared: $it") }
    stateFlow.collect { println("State: $it") }
}

// 발행
CoroutineScope(Dispatchers.IO).launch {
    sharedFlow.emit("Shared message")
    stateFlow.value = "New state"
}
```

In [20]:
%use coroutines

In [21]:
runBlocking {
    println("Hello, World!")
    delay(1000L)
    println("Goodbye, World!1")
}

Hello, World!
Goodbye, World!1


In [22]:
// flatMapLatest 사용해보기
// flatMapLatest 사용하여 flow를 변환하는 예제
// upstream 과 downstream으로 나눠서 고민
// switchMap 같음.
// -----1-----------------2-----------------3------------------
// -----A-----B-----C-----A-----B-----C-----A-----B-----C
// 1->A
// 1->B
// 1->C
// 2->A
// 2->B
// 2->C
// 3->A
// 3->B
// 3->C
runBlocking {
    val upstream = flowOf(1, 2, 3)
    val downstream = flowOf("A", "B", "C")

    val result = upstream.flatMapLatest { value ->
        downstream.map { "$value -> $it" }
    }

    result.collect { println(it) }
}

1 -> A
1 -> B
1 -> C
2 -> A
2 -> B
2 -> C
3 -> A
3 -> B
3 -> C
